# Introduction to Property Packages in IDAES

In [1]:
# Important components from Pyomo
from pyomo.environ import (
    Constraint,
    Expression,
    Reference,
    Param,
    units as pyunits,
    Var,
)

# Import IDAES cores
from idaes.core import (
    declare_process_block_class,
    MaterialFlowBasis,
    PhysicalParameterBlock,
    StateBlockData,
    StateBlock,
    MaterialBalanceType,
    EnergyBalanceType,
    Component,
    VaporPhase,
)
from idaes.core.solvers import get_solver
from idaes.core.util.initialization import (
    fix_state_vars,
    revert_state_vars,
    solve_indexed_blocks,
)
from idaes.core.util.model_statistics import (
    degrees_of_freedom,
    number_unfixed_variables,
)
from idaes.core.util.constants import Constants as constants
import idaes.logger as idaeslog

# Step 1: Define Units of Measurement

In [2]:
units_metadata = {
    "time": pyunits.s,
    "length": pyunits.m,
    "mass": pyunits.kg,
    "amount": pyunits.mol,
    "temperature": pyunits.K,
}

# Step 2: Define Supported Properties

In [3]:
properties_metadata = {
    "flow_mol": {"method": None},
    "mole_frac_comp": {"method": None},
    "temperature": {"method": None},
    "pressure": {"method": None},
    "mw_comp": {"method": None},
    "dens_mol": {"method": None},
    "enth_mol": {"method": "_enth_mol"},
}

# Step 3: Define Component and Phase Lists

In [4]:
def define_components_and_phases(self):
    # Define Component objects for all species
    self.benzene = Component()
    self.toluene = Component()
    self.methane = Component()
    self.hydrogen = Component()
    self.diphenyl = Component()

    # Define Phase objects for all phases
    self.Vap = VaporPhase()

# Step 4: Define Parameters


In [6]:
def define_basic_parameters(self):
    # Thermodynamic reference state
    self.pressure_ref = Param(
        mutable=True, default=101325, units=pyunits.Pa, doc="Reference pressure"
    )
    self.temperature_ref = Param(
        mutable=True, default=298.15, units=pyunits.K, doc="Reference temperature"
    )

    self.mw_comp = Param(
        self.component_list,
        mutable=False,
        initialize={
            "benzene": 78.1136e-3,
            "toluene": 92.1405e-3,
            "hydrogen": 2.016e-3,
            "methane": 16.043e-3,
            "diphenyl": 154.212e-4,
        },
        units=pyunits.kg / pyunits.mol,
        doc="Molecular weight",
    )

### Correlation for calculating the specific enthalpy for each component

\begin{equation*} h_j – h_{j, ref}= A_j \times (T-T_{ref}) + \frac{B_j}{2}\times (T^2-T_{ref}^2) + \frac{C_j}{3}\times (T^3-T_{ref}^3) + \frac{D_j}{4}\times (T^4-T_{ref}^4) \end{equation*}

where $h_{j,ref}$ is the standard heat of formation of component $j$ in the vapor phase, and $A_j$, $B_j$, $C_j$, and $D_j$ are component-specific parameters in the correlation.

In [7]:
def define_specific_heat_parameters(self):
    # Constants for specific heat capacity, enthalpy
    self.cp_mol_ig_comp_coeff_A = Var(
        self.component_list,
        initialize={
            "benzene": -3.3921e1,
            "toluene": -2.435e1,
            "hydrogen": 2.714e1,
            "methane": 1.925e1,
            "diphenyl": -9.707e1,
        },
        units=pyunits.J / pyunits.mol / pyunits.K,
        doc="Parameter A for ideal gas molar heat capacity",
    )
    self.cp_mol_ig_comp_coeff_A.fix()

    self.cp_mol_ig_comp_coeff_B = Var(
        self.component_list,
        initialize={
            "benzene": 4.739e1,
            "toluene": 5.125e-1,
            "hydrogen": 9.274e-3,
            "methane": 5.213e-2,
            "diphenyl": 1.106e0,
        },
        units=pyunits.J / pyunits.mol / pyunits.K**2,
        doc="Parameter B for ideal gas molar heat capacity",
    )
    self.cp_mol_ig_comp_coeff_B.fix()

    self.cp_mol_ig_comp_coeff_C = Var(
        self.component_list,
        initialize={
            "benzene": -3.017e-4,
            "toluene": -2.765e-4,
            "hydrogen": -1.381e-5,
            "methane": -8.855e-4,
            "diphenyl": -8.855e-4,
        },
        units=pyunits.J / pyunits.mol / pyunits.K**3,
        doc="Parameter C for ideal gas molar heat capacity",
    )
    self.cp_mol_ig_comp_coeff_C.fix()

    self.cp_mol_ig_comp_coeff_D = Var(
        self.component_list,
        initialize={
            "benzene": 7.130e-8,
            "toluene": 4.911e-8,
            "hydrogen": 7.645e-9,
            "methane": -1.132e-8,
            "diphenyl": 2.790e-7,
        },
        units=pyunits.J / pyunits.mol / pyunits.K**4,
        doc="Parameter D for ideal gas molar heat capacity",
    )
    self.cp_mol_ig_comp_coeff_D.fix()

    self.enth_mol_form_vap_comp_ref = Var(
        self.component_list,
        initialize={
            "benzene": -82.9e3,
            "toluene": -50.1e3,
            "hydrogen": 0,
            "methane": -75e3,
            "diphenyl": -180e3,
        },
        units=pyunits.J / pyunits.mol,
        doc="Standard heat of formation at reference state",
    )
    self.enth_mol_form_vap_comp_ref.fix()

## Declaring the Physical Parameter Block

In [8]:
@declare_process_block_class("HDAParameterBlock")
class HDAParameterData(PhysicalParameterBlock):
    CONFIG = PhysicalParameterBlock.CONFIG()

    def build(self):
        """
        Callable method for Block construction
        """
        super(HDAParameterData, self).build()

        self._state_block_class = HDAStateBlock

        define_components_and_phases(self)
        define_basic_parameters(self)
        define_specific_heat_parameters(self)

    @classmethod 
    def define_metadata(cls, obj):
        """
        Define properties supported and units.
        """
        obj.add_properties(properties_metadata)

        obj.add_default_units(units_metadata)

# Step 5: Declare State Variables

In [9]:
def add_state_variables(self):
    self.flow_mol = Var(
        initialize=1,
        bounds=(1e-8, 1000),
        units=pyunits.mol / pyunits.s,
        doc="Molar flow rate",
    )
    self.mole_frac_comp = Var(
        self.component_list,
        initialize=0.2,
        bounds=(0, None),
        units=pyunits.dimensionless,
        doc="Component mole fractions",
    )
    self.pressure = Var(
        initialize=101325,
        bounds=(101325, 400000),
        units=pyunits.Pa,
        doc="State pressure",
    )
    self.temperature = Var(
        initialize-298.15,
        bounds=(298.15, 1500),
        units=pyunits.K,
        doc="State temperature"
    ) 

In [10]:
def return_state_var_dict(self):
    return {
        "flow_mol": self.flow_mol,
        "mole_frac_comp": self.mole_frac_comp,
        "temperature": self.temperature,
        "pressure": self.pressure,
    }

# Step 6: Write Constraints and/or Expressions for Properties of Interest

In [11]:
def add_molecular_weight_and_density(self):
    self.mw_comp = Reference(self.params.mw_comp)

    self.dens_mol = Var(
        initialize=1, units=pyunits.mol / pyunits.m**3, doc="Mixture density"
    )

    self.ideal_gas_eq = Constraint(
        expr=self.pressure == const.gas_constant * self.temperature * self.dens_mol
    )

The last property of interest we need to declare is the mixture specific enthalpy. For an ideal gas, the mixture specific enthalpy can be calculated from the component specific enthalpies using the following equation:

\begin{equation*} 
h_{mix}= \sum{x_j \times h_j} 
\end{equation*}

where $x_j$ is the mole fraction of component $j$. Recall that for this example we are using the following correlation for the component specific enthalpies:

\begin{equation*} 
h_j – h_{j, ref}= A_j \times (T-T_{ref}) + \frac{B_j}{2}\times (T^2-T_{ref}^2) + \frac{C_j}{3}\times (T^3-T_{ref}^3) + \frac{D_j}{4}\times (T^4-T_{ref}^4) 
\end{equation*}

In [12]:
def add_enth_mol(self):
    def enth_rule(b):
        params = self.params
        T = self.temperature
        Tr = params.temperature_ref
        return sum(
            self.mole_frac_comp[j]
            * (
                (params.cp_mol_ig_comp_coeff_D[j] / 4) * (T**4 - Tr**4)
                + (params.cp_mol_ig_comp_coeff_C[j] / 3) * (T**3 - Tr**3)
                + (params.cp_mol_ig_comp_coeff_B[j] / 2) * (T**2 - Tr**2)
                + params.cp_mol_ig_comp_coeff_A[j] * (T - Tr)
                + params.enth_mol_form_vap_comp_ref[j]
            )
            for j in self.component_list
        )

    self.enth_mol = Expression(rule=enth_rule)

In [13]:
def add_mole_fraction_constraint(self):
    if self.config.defined_state is False:
        self.mole_fraction_constraint = Constraint(
            expr=1e3 == sum(1e3 * self.mole_frac_comp[j] for j in self.component_list)
        )

# Step 7: Define an Initialization Routine

In [14]:
def prepare_state(blk, state_args, state_vars_fixed):
    # Fix state variables if not already fixed
    if state_vars_fixed is False:
        flags = fix_state_vars(blk, state_args)
    else:
        flags = None

    # Deactivate sum of mole fractions constraint
    for k in blk.keys():
        if blk[k].config.defined_state is False:
            blk[k].mole_fraction_constraint.deactivate()

    # Check that degress of freedom are zero after fixing state vars
    for k in blk.keys():
        if degrees_of_freedom(blk[k]) != 0:
            raise Exception(
                "State vars fixed but degrees of freedom"
                "for state block is not zero during"
                "initialization."
            )
    
    return flags

In [16]:
def initialize_state(blk, solver, init_log, solve_log):
    # Check that there is something to solve for
    free_vars = 0
    for k in blk.keys():
        free_vars += number_unifxed_variables(blk[k])
    if free_vars > 0:
        # If there are free variables, call the solver to initialize
        try:
            with idaeslog.solver_log(solve_log, idaeslog.DEBUG) as slc:
                res = solve_indexed_blocks(solver, [blk], tee=True) # slc.tee
        except:
            res = None
    else:
        res = None

    init_log.info("Properties Initialized {}.".formate(idaeslog.condition(res)))
        

In [17]:
def restore_state(blk, flags, hold_state):
    # Return state to initial conditions
    if hold_state is True:
        return flags
    else:
        blk.release_state(flags)

In [18]:
def unfix_state(blk, flags, outlvl):
    init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")

    # Reactivate sum of mole fractions constraint
    for k in blk.keys():
        if blk[k].config.defined_state is False:
            blk[k].mole_fraction_constraint.activate()

    if flags is not None:
        # Unfix state variables
        revert_state_vars(blk, flags)

    init_log.info_high("State Released.")

## The StateBlock class

In [20]:
class _HDAStateBlock(StateBlock):
    def initialize(
        blk,
        state_args=None,
        state_vars_fixed=False,
        hold_state=False,
        outlvl=idaeslog.NOTSET,
        solver=None,
        optarg=None,
    ):

        init_log = idaeslog.getInitLogger(blk.name, outlvl, tag="properties")
        solve_log = idaeslog.getSolverLogger(blk.name, outlvl, tag="properties")

        # Create solver
        solver_obj = get_solver(solver, optarg)

        flags = prepare_state(blk, state_args, state_vars_fixed)
        initialize_state(blk, solver_obj, init_log, solve_log)
        restore_state(blk, flags, hold_state)

        init_log.info("Initialization Complete")

    def release_state(blk, flags, outlvl=idaeslog.NOTSET):
        unfix_state(blk, flags, outlvl)